# BERT 预训练

# 1 训练数据读取

# 参考 https://github.com/google-research/bert
# 数据集：https://modelscope.cn/datasets/AI-ModelScope/chinese-c4
# paper : https://arxiv.org/pdf/1810.04805
# 本代码git: https://github.com/zhangjianzoujianghu18/learning-gpt.git

### The masked language model randomly masks some of the tokens from the input, and the objective is to predict the original vocabulary id of the masked word based only on its context


In [8]:
import zstandard as zstd
import json
import io
import os
import glob
from datasets import load_dataset, Dataset, IterableDataset

data_path_root = "/Users/wangaijun/pythoncode/github/data/text"

# 查找所有 .jsonl.zst 文件
jsonl_zst_files = glob.glob(f'{data_path_root}/chinese-c4/*.jsonl.zst', recursive=True)

# 定义一个生成器函数来读取 .jsonl.zst 文件
def read_jsonl_zst(file_path):
    with open(file_path, 'rb') as fh:
        dctx = zstd.ZstdDecompressor()
        stream_reader = dctx.stream_reader(fh)
        text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8')
        for line in text_stream:
            yield json.loads(line)

# 定义一个生成器函数来读取所有 .jsonl.zst 文件
def read_all_jsonl_zst(files):
    for file_path in files:
        yield from read_jsonl_zst(file_path)

# 使用 HuggingFace 的 IterableDataset 来流式加载数据
dataset = IterableDataset.from_generator(
    lambda: read_all_jsonl_zst(jsonl_zst_files)
)

# 打印前几条数据以验证
for i, example in enumerate(dataset):
    print(example)
    if i >= 2:  # 打印前3条数据
        break

{'url': 'http://sports.sina.com.cn/c/2011-01-05/08555394624.shtml', 'timestamp': '2022-12-08T23:26:29Z', 'content_language': 'zho,eng', 'content_type': 'text/plain', 'text': '尽管淡出中国足坛好多年，但“龙哥”还是个在亚足联甚至国际足联都举足轻重的人物，他现在头上那个“中国足协副主席”的头衔在国内属于前朝遗物，他目前是体育总局对外体育交流中心主任、党委副书记，但亚足联第一副主席这个顶戴却是货真价实的，此番前往西亚，张吉龙是想更上一层楼，拿下国际足联执委的位置。\n本次竞选地被安排在多哈喜来登大酒店，在通过一系列决议后，竞选将从当地时间6日下午开始。竞选顺序依次为国际足联副主席竞选、国际足联执委竞选、亚足联副主席竞选及亚足联其他要职的换届选举，亚足联每个会员协会拥有一张选票，竞选以简单多数为原则，即得票多者获胜，得票无需过半数。\n根据国际足联和亚足联规定，参加国际足联副主席竞选的韩国人郑梦准、约旦阿里侯赛因王子两人之间的获胜者将自动获得下一届国际足联执委资格。按照亚足联以往竞选惯例，东亚区竞选人一旦竞得国际足联副主席之位，那么该地区在接下来的国际足联执委竞选中很可能陷入不利境地。毕竟在亚足联东西亚势力博弈的背景下，在亚足联45个正式会员协会中占据绝对比重的西亚足球不希望东亚兼得国际足联副主席和执委两大要职。而中国首轮若投给郑梦准，那么张吉龙在竞选国际足联执委时很可能将失去有利地位；若投给侯赛因王子，那么对东亚足球的整体利益也会造成间接伤害。\n足协一位相关人士表示：“韦主任把票投给谁，代表团还要商议，他投票原则是确保中国足球利益最大化，就像去年5月中国足协将票意外投给哈曼一样。”而韦迪提前3天抵达多哈，其目的就是与亚足联和各会员协会进行游说公关，确保张吉龙能够成功当选国际足联执委。\n张吉龙在阎世铎当政后期逐渐在中国足协“失宠”，从而形成一个奇特的现象――这位在国际足坛最负盛名的中国足球官员，却被调离了足管中心，虽然在2008年奥运会后确认为正厅级足管中心主任，但却是为他调往对外交流中心做铺垫，所以他现在头上所戴的足协副主席帽子，仅仅是因为足协已经很长时间没有改选的原因。而且，他在亚足联的位置，

In [9]:
example["text"]

'中国伟大的无产阶级革命家、政治家、军事家，中华人民共和国元帅、中国人民解放军的缔造者之一。1897年4月28日生于广东省梅县雁洋堡。少年时在丙村三堡学堂和梅县东山中学读书，受到辛亥革命影响，立志报效国家。1916年随父赴南洋。翌年回国，入云南讲武堂学习，毕业后追随孙中山先生，投身于民主革命。1920年夏，叶剑英参加了孙中山组织的驱逐桂系军阀之役。翌年10月，随大总统孙中山出巡广西。1922年6月，军阀陈炯明叛变。任海军陆战队营长的叶剑英率部护卫孙中山脱险，同叛军英勇作战。嗣后，前往福建任东路讨贼军第八旅参谋长，随军入粤讨伐陈炯明。1924年初，叶剑英任建国粤军第二师参谋长。受廖仲恺先生邀请，参加创建黄埔陆军军官学校，任教授部副主任，是当时很有威望的教官之一。这时，他接受了马克思列宁主义并要求加入中国共产党。\n1925年，叶剑英参加讨伐陈炯明的两次东征。英勇善战，指挥果断，显露了出众的军事才能。1926年7月，参加北伐战争。初期任国民革命军第一军总预备队指挥部参谋长。攻克南昌后，任国民革命军新编第二师师长。1927年4月12日，蒋介石发动反革命政变。叶剑英毅然通电反蒋。随即奔赴武汉，任国民革命军第四军参谋长。7月，在严重的白色恐怖中秘密加入中国共产党。\n1927年南昌起义前，叶剑英得知汪精卫阴谋加害叶挺、贺龙，立即冒着风险找他们商量对策，决定叶挺和贺龙指挥的部队迅速向南昌开进，使敌人阴谋没有得逞，南昌起义得以实现。起义军撤出南昌后，叶剑英极力劝阻了张发奎对起义军的追击。8月上旬，兼任第四军军官教导团团长，使这支革命武装成为广州起义的主力。他促成警卫团扩编，介绍共产党员梁秉枢当团长，使警卫团也成为广州起义的武装力量。12月11日，他与张太雷、叶挺等领导了广州起义，任工农红军副总指挥。这次起义，和南昌起义、秋收起义相连接，成为第二次国内革命战争与创立中国工农红军的伟大开端。\n1928年冬，叶剑英赴苏联共产主义劳动大学特别班学习，1930年下半年回国。1931年初到达中央苏区，历任中央革命军事委员会委员兼总参谋部部长(即总参谋长)，红一方面军参谋长，闽赣军区、福建军区司令员等职务，参与指挥第二、三、四次反“围剿”战役。在此期间，还担任中国工农红军学校校长兼政治委员，培养了大批军政干部。\n1934年10月，中央红军进行长征。叶剑英任军委第一纵队司令员。部队进入广西山

# 2 documents加工，数据加工成文档

In [10]:
from transformers import BertTokenizer
import random
import numpy as np
tokenizer = BertTokenizer.from_pretrained('/Users/wangaijun/pythoncode/github/model/bert-base-chinese')
vocab_words = list(tokenizer.vocab.keys())
print(len(vocab_words))
rng = random.Random(22)
# print("tokenizer 长度：",len(tokenizer))
# print("tokenizer ",tokenizer.tokenize(text))
# 打印前几条数据以验证
all_documents = [[]]
for i, example in enumerate(dataset):
    text=example["text"]
    tokens = tokenizer.tokenize(text)
    all_documents.append(tokens)
    if i >= 2:  # 打印前条5数据
        break
all_documents = [x for x in all_documents if x]
# rng.shuffle(all_documents)
len(all_documents)

21128


3

In [5]:
vocab_words

NameError: name 'vocab_words' is not defined

# 3 文档切分成训练语料

### [CLS] is a special symbol added in front of every input example, and [SEP] is a special separator token

### next sentence : 50% of the time B is the actual next sentence that follows A (labeled as IsNext), and 50% of the time it is a random sentence from the corpus (labeled as NotNext).

<div style="text-align: center;">
  <img src="images/bert_mask_sen.png" alt="Image" style="width:600px;">
</div>


In [26]:
max_seq_length=512
short_seq_prob=0.2
sentences=[]

for document_index,doc in enumerate(all_documents):
    doc_len=len(doc)
    max_num_tokens = max_seq_length - 3  #[CLS, SEQ,SEQ]
    target_seq_length=max_num_tokens
    if rng.random() < short_seq_prob:
        target_seq_length = rng.randint(2, max_num_tokens)
    a_start=0 
    while a_start<=doc_len-4:
        tokens=[] 
        segment_ids = []
        is_next_sentence=0
        a_length = rng.randint(1, target_seq_length - 1) # a 句子长度随机取
        a_end=a_start+a_length
        a_sentence=doc[a_start:a_end]
        b_length=target_seq_length-a_length # b的长度为句子长度- a句子长度
        if b_length== 1 or rng.random() < 0.5: # B句子是否来自下一句,有50%概率来自下一句
            for _ in range(10): # 从其他文档中随机选择一个文档，里的句子作为一个句子
                random_document_index = rng.randint(0, len(all_documents) - 1)
                if random_document_index != document_index:
                    break 
            random_document_next = all_documents[random_document_index]
            random_start = rng.randint(0, len(random_document_next) - 1)
            b_sentence=random_document_next[random_start:random_start+b_length]
            a_start=a_end # 下一次的a_start 从a_end开始
            is_next_sentence=0
        else:
            b_end=min(a_end+b_length,doc_len)
            b_sentence=doc[a_end:a_end+b_length]
            # 下一次的a_start 从a_length+b_length开始
            a_start+=target_seq_length
            is_next_sentence=1
        if  len(a_sentence) >= 1 and  len(b_sentence) >= 1:
            tokens.append("[CLS]")
            segment_ids.append(0)
            tokens.extend(a_sentence)
            [segment_ids.append(0) for i in a_sentence]
            tokens.append("[SEP]")
            segment_ids.append(0)
            
            tokens.extend(b_sentence)
            [segment_ids.append(1) for i in b_sentence]
            tokens.append("[SEP]")
            segment_ids.append(1)
            sentences.append((tokens,segment_ids,is_next_sentence))
print("".join(sentences[-1][0]))
print("is next sentence ",sentences[-1][2])
print("".join([str(i) for i in sentences[-1][1]]))

[CLS]委员会委员。1983年2月25日，在第五届全国人民代表大会任期将满的时候，叶剑英写信给全国人大常委会，请求不再提名选他当六届全国人大代表，不再将他列为六届全国人大常委会委员长候选人。3月5日，人大常委会复信同意他的要求，热情地赞扬了他在长达半个多世纪的充满艰难险阻的革命斗争中所建立的丰功伟绩，赞扬他为社会主义建设事业和为实现祖国统一大业作出的卓越贡献。6月，在第六届全国人大第一次会议上，他被任命为中华人民共和国中央军事委员会副主席。10月，参与主持党的十二届二中全会。这一年，出版了《叶剑英诗词选集》。他在晚年，还多次与王震等老战友一起，到全国各地视察，关心和支持一线同志的工作。1985年9月，叶剑英同其他六十三位老同志一起致函党的十二届四中全会，请求不再担任中央委员，以便让一批比较年轻的德才兼备的同志进入中央委员会，进一步实现中央领导机构成员的新老交替。十二届四中全会同意他的请求，并给他写了致敬信，高度评价了他的光辉业绩。1986年10月22日因病在北京逝世，享年89岁。[SEP]上述专利申请权或专利权转让的著录项目变更手续，必须由转让方的申请人或专[SEP]
is next sentence  0
0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000111111111111111111111111111111111111


# 4 随机mask句子

# 1 we simply mask some percentage of the input tokens at random, and then predict those masked tokens
##  1️⃣ The training data generator chooses 15% of the token positions at random for  prediction.
##  2️⃣ with (1) the [MASK] token 80% of the time
##  3️⃣ a random token 10% of the time
##  4️⃣ unchanged i-th token 10% of the time.


In [27]:
max_predictions_per_seq=20
masked_lm_prob=0.15
for tokens,segment_ids,is_next_sentence in sentences:
    cand_indexes = []
    for (i, token) in enumerate(tokens):
        if token == "[CLS]" or token == "[SEP]":
            continue
        cand_indexes.append(i)
    rng.shuffle(cand_indexes)
    
    output_tokens = list(tokens)
    num_to_predict = min(max_predictions_per_seq, max(1, int(round(len(tokens) * masked_lm_prob))))

    masked_lms=[]
    for num in range(num_to_predict):
        index=cand_indexes[num]
        masked_token = None
        if rng.random() < 0.8:
            masked_token = "[MASK]"
        else:
            if rng.random() < 0.5:
                masked_token = tokens[index]
            else:
                masked_token = vocab_words[rng.randint(0, len(vocab_words) - 1)]

        output_tokens[index] = masked_token

        masked_lms.append((index, tokens[index]))
        
    masked_lms = sorted(masked_lms, key=lambda x: x[0])
    masked_lm_positions = []
    masked_lm_labels = []
    for p in masked_lms:
#         print("p",p)
        masked_lm_positions.append(p[0])
        masked_lm_labels.append(p[1])
    break
masked_lms,''.join(output_tokens)

([(8, '坛'),
  (39, '，'),
  (49, '国'),
  (60, '国'),
  (90, '合'),
  (95, '我'),
  (109, '件'),
  (204, '建'),
  (221, '泽'),
  (231, '后'),
  (240, '迅'),
  (293, '任'),
  (309, '书'),
  (312, '主'),
  (371, '来'),
  (438, '次'),
  (439, '来'),
  (455, '外'),
  (486, '为'),
  (495, '年')],
 '[CLS]尽管淡出中国足[MASK]好多年，但[UNK]龙哥[UNK]还是个在亚足联甚至国际足联都举足轻重的人物##姹他现在头上那个[UNK]中[MASK]足协副主席[UNK]的头衔在[MASK]内属于前朝遗物，他目前是[SEP]；主张军事训练和军事科学研究相结[MASK]，以总结[MASK]军的经验为主，探讨在现代条件下的战争指导规律；主持制定人民解放军一系列条令条例；参与领导军事科学技术的发展工作，为把我军建设成强大的现代化正规化革命军队，为坚持和发展毛泽东军事思想，作出了多方面建树。1963年12月，叶剑英[MASK]议在全军推广郭兴福教学法，得到毛[MASK]东和军委的赞许。之[MASK]，群众性练兵热潮[MASK]速地在全军开展起来，取得显著成果。1965年，任第四届政协全国委员会副主席。1966年1月，任中央军委副主席。5月，[MASK]中共中央书记处书记、中央军委秘[MASK]长，涂持军委日常工作。8月，在党的八届十一中全会上当选为中央政治局委员。1971年7月，叶剑英受毛泽东、周恩来委托，主持接待秘密来华访问的美国总统国家安全事务助理基辛格。1972年参加接待先后来华访问的美国总统尼克松和日本国总理大臣田中角荣。1973年，又协助周恩来接待第二[MASK][MASK]访的基辛格。他为建立中美、中日[MASK]交关系作出了重要贡献。1973年8月，在党的十届一中全会上，他当选⁺党中央副主席。1974[MASK]1月，叶剑英受毛泽东、周恩来委[SEP]')

In [4]:
vocab_words

NameError: name 'vocab_words' is not defined

# 5 mask 构造成示例进行存储

In [28]:
class TrainingInstance:
    def __init__(self, tokens, segment_ids, masked_lm_positions, masked_lm_labels, is_random_next):
        self.tokens = tokens
        self.segment_ids = segment_ids
        self.is_random_next = is_random_next
        self.masked_lm_positions = masked_lm_positions
        self.masked_lm_labels = masked_lm_labels

    def __str__(self):
        s = ""
        s += "tokens: %s\n" % (" ".join(self.tokens))
        s += "segment_ids: %s\n" % (" ".join(map(str, self.segment_ids)))
        s += "is_random_next: %s\n" % self.is_random_next
        s += "masked_lm_positions: %s\n" % (" ".join(map(str, self.masked_lm_positions)))
        s += "masked_lm_labels: %s\n" % (" ".join(self.masked_lm_labels))
        s += "\n"
        return s

    def __repr__(self):
        return self.__str__()
		

In [29]:
instances=[]
instance=TrainingInstance(output_tokens,segment_ids,masked_lm_positions,masked_lm_labels,is_next_sentence)
instances.append(instance)
instance

tokens: [CLS] 尽 管 淡 出 中 国 足 [MASK] 好 多 年 ， 但 [UNK] 龙 哥 [UNK] 还 是 个 在 亚 足 联 甚 至 国 际 足 联 都 举 足 轻 重 的 人 物 ##姹 他 现 在 头 上 那 个 [UNK] 中 [MASK] 足 协 副 主 席 [UNK] 的 头 衔 在 [MASK] 内 属 于 前 朝 遗 物 ， 他 目 前 是 [SEP] ； 主 张 军 事 训 练 和 军 事 科 学 研 究 相 结 [MASK] ， 以 总 结 [MASK] 军 的 经 验 为 主 ， 探 讨 在 现 代 条 件 下 的 战 争 指 导 规 律 ； 主 持 制 定 人 民 解 放 军 一 系 列 条 令 条 例 ； 参 与 领 导 军 事 科 学 技 术 的 发 展 工 作 ， 为 把 我 军 建 设 成 强 大 的 现 代 化 正 规 化 革 命 军 队 ， 为 坚 持 和 发 展 毛 泽 东 军 事 思 想 ， 作 出 了 多 方 面 建 树 。 1963 年 12 月 ， 叶 剑 英 [MASK] 议 在 全 军 推 广 郭 兴 福 教 学 法 ， 得 到 毛 [MASK] 东 和 军 委 的 赞 许 。 之 [MASK] ， 群 众 性 练 兵 热 潮 [MASK] 速 地 在 全 军 开 展 起 来 ， 取 得 显 著 成 果 。 1965 年 ， 任 第 四 届 政 协 全 国 委 员 会 副 主 席 。 1966 年 1 月 ， 任 中 央 军 委 副 主 席 。 5 月 ， [MASK] 中 共 中 央 书 记 处 书 记 、 中 央 军 委 秘 [MASK] 长 ， 涂 持 军 委 日 常 工 作 。 8 月 ， 在 党 的 八 届 十 一 中 全 会 上 当 选 为 中 央 政 治 局 委 员 。 1971 年 7 月 ， 叶 剑 英 受 毛 泽 东 、 周 恩 来 委 托 ， 主 持 接 待 秘 密 来 华 访 问 的 美 国 总 统 国 家 安 全 事 务 助 理 基 辛 格 。 1972 年 参 加 接 待 先 后 来 华 访 问 的 美 国 总 统 尼 克 松 和 日 本 国 总 理 大 臣 田 中 角 荣 。 1973 年 ， 又 协 助 周 恩 来 接 待 第 二 [MASK] [MASK] 访 

# 6 数据写入本地

In [30]:
all_features = []
for (inst_index, instance) in enumerate(instances):
    input_ids = tokenizer.convert_tokens_to_ids(instance.tokens)
    print(" ".join(map(str, input_ids)))
    input_mask = [1] * len(input_ids)
    segment_ids = list(instance.segment_ids)

    assert len(input_ids) <= max_seq_length

    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    masked_lm_positions = list(instance.masked_lm_positions)
    masked_lm_ids = tokenizer.convert_tokens_to_ids(instance.masked_lm_labels)
    masked_lm_weights = [1.0] * len(masked_lm_ids)

    while len(masked_lm_positions) < max_predictions_per_seq:
        masked_lm_positions.append(0)
        masked_lm_ids.append(0)
        masked_lm_weights.append(0.0)

    next_sentence_label = 1 if instance.is_random_next else 0

    features = {
        "input_ids": input_ids,
        "input_mask": input_mask,
        "segment_ids": segment_ids,
        "masked_lm_positions": masked_lm_positions,
        "masked_lm_ids": masked_lm_ids,
        "masked_lm_weights": masked_lm_weights,
        "next_sentence_labels": next_sentence_label
    }

    all_features.append(features)
features
# for output_file in output_files:
#     with open(output_file, "w", encoding="utf-8") as writer:
#         for feature in all_features:
#             writer.write(json.dumps(feature, ensure_ascii=False) + "\n")

101 2226 5052 3909 1139 704 1744 6639 103 1962 1914 2399 8024 852 100 7987 1520 100 6820 3221 702 1762 762 6639 5468 4493 5635 1744 7354 6639 5468 6963 715 6639 6768 7028 4638 782 4289 15068 800 4385 1762 1928 677 6929 702 100 704 103 6639 1291 1199 712 2375 100 4638 1928 6124 1762 103 1079 2247 754 1184 3308 6890 4289 8024 800 4680 1184 3221 102 8039 712 2476 1092 752 6378 5298 1469 1092 752 4906 2110 4777 4955 4685 5310 103 8024 809 2600 5310 103 1092 4638 5307 7741 711 712 8024 2968 6374 1762 4385 807 3340 816 678 4638 2773 751 2900 2193 6226 2526 8039 712 2898 1169 2137 782 3696 6237 3123 1092 671 5143 1154 3340 808 3340 891 8039 1346 680 7566 2193 1092 752 4906 2110 2825 3318 4638 1355 2245 2339 868 8024 711 2828 2769 1092 2456 6392 2768 2487 1920 4638 4385 807 1265 3633 6226 1265 7484 1462 1092 7339 8024 711 1780 2898 1469 1355 2245 3688 3813 691 1092 752 2590 2682 8024 868 1139 749 1914 3175 7481 2456 3409 511 9155 2399 8110 3299 8024 1383 1187 5739 103 6379 1762 1059 1092 2972 

{'input_ids': [101,
  2226,
  5052,
  3909,
  1139,
  704,
  1744,
  6639,
  103,
  1962,
  1914,
  2399,
  8024,
  852,
  100,
  7987,
  1520,
  100,
  6820,
  3221,
  702,
  1762,
  762,
  6639,
  5468,
  4493,
  5635,
  1744,
  7354,
  6639,
  5468,
  6963,
  715,
  6639,
  6768,
  7028,
  4638,
  782,
  4289,
  15068,
  800,
  4385,
  1762,
  1928,
  677,
  6929,
  702,
  100,
  704,
  103,
  6639,
  1291,
  1199,
  712,
  2375,
  100,
  4638,
  1928,
  6124,
  1762,
  103,
  1079,
  2247,
  754,
  1184,
  3308,
  6890,
  4289,
  8024,
  800,
  4680,
  1184,
  3221,
  102,
  8039,
  712,
  2476,
  1092,
  752,
  6378,
  5298,
  1469,
  1092,
  752,
  4906,
  2110,
  4777,
  4955,
  4685,
  5310,
  103,
  8024,
  809,
  2600,
  5310,
  103,
  1092,
  4638,
  5307,
  7741,
  711,
  712,
  8024,
  2968,
  6374,
  1762,
  4385,
  807,
  3340,
  816,
  678,
  4638,
  2773,
  751,
  2900,
  2193,
  6226,
  2526,
  8039,
  712,
  2898,
  1169,
  2137,
  782,
  3696,
  6237,
  3123,
  1092

# 7 合并处理过程

## 7.1 读取数据

In [11]:
from transformers import BertTokenizer
import random
import numpy as np
# 打印前几条数据以验证
for i, example in enumerate(dataset):
    print(example)
    if i >= 2:  # 打印前3条数据
        break

{'url': 'http://sports.sina.com.cn/c/2011-01-05/08555394624.shtml', 'timestamp': '2022-12-08T23:26:29Z', 'content_language': 'zho,eng', 'content_type': 'text/plain', 'text': '尽管淡出中国足坛好多年，但“龙哥”还是个在亚足联甚至国际足联都举足轻重的人物，他现在头上那个“中国足协副主席”的头衔在国内属于前朝遗物，他目前是体育总局对外体育交流中心主任、党委副书记，但亚足联第一副主席这个顶戴却是货真价实的，此番前往西亚，张吉龙是想更上一层楼，拿下国际足联执委的位置。\n本次竞选地被安排在多哈喜来登大酒店，在通过一系列决议后，竞选将从当地时间6日下午开始。竞选顺序依次为国际足联副主席竞选、国际足联执委竞选、亚足联副主席竞选及亚足联其他要职的换届选举，亚足联每个会员协会拥有一张选票，竞选以简单多数为原则，即得票多者获胜，得票无需过半数。\n根据国际足联和亚足联规定，参加国际足联副主席竞选的韩国人郑梦准、约旦阿里侯赛因王子两人之间的获胜者将自动获得下一届国际足联执委资格。按照亚足联以往竞选惯例，东亚区竞选人一旦竞得国际足联副主席之位，那么该地区在接下来的国际足联执委竞选中很可能陷入不利境地。毕竟在亚足联东西亚势力博弈的背景下，在亚足联45个正式会员协会中占据绝对比重的西亚足球不希望东亚兼得国际足联副主席和执委两大要职。而中国首轮若投给郑梦准，那么张吉龙在竞选国际足联执委时很可能将失去有利地位；若投给侯赛因王子，那么对东亚足球的整体利益也会造成间接伤害。\n足协一位相关人士表示：“韦主任把票投给谁，代表团还要商议，他投票原则是确保中国足球利益最大化，就像去年5月中国足协将票意外投给哈曼一样。”而韦迪提前3天抵达多哈，其目的就是与亚足联和各会员协会进行游说公关，确保张吉龙能够成功当选国际足联执委。\n张吉龙在阎世铎当政后期逐渐在中国足协“失宠”，从而形成一个奇特的现象――这位在国际足坛最负盛名的中国足球官员，却被调离了足管中心，虽然在2008年奥运会后确认为正厅级足管中心主任，但却是为他调往对外交流中心做铺垫，所以他现在头上所戴的足协副主席帽子，仅仅是因为足协已经很长时间没有改选的原因。而且，他在亚足联的位置，

In [32]:
## 7.2 读取文档和分词

In [13]:
tokenizer = BertTokenizer.from_pretrained('/Users/wangaijun/pythoncode/github/model/bert-base-chinese')
vocab_words = list(tokenizer.vocab.keys())
print(len(vocab_words))
rng = random.Random(22)
all_documents = []
for i, example in enumerate(dataset):
    text=example["text"]
    tokens = tokenizer.tokenize(text)
    all_documents.append(tokens)
    if i >= 10:  # 打印前条5数据
        break
all_documents = [x for x in all_documents if x]
# rng.shuffle(all_documents)
len(all_documents)

21128


11

In [14]:
# 7.3 文档加工成训练句子
max_seq_length=512
short_seq_prob=0.2
def getTrainSentenceFromDoc(all_documents):
    sentences=[]
    for document_index,doc in enumerate(all_documents):
        doc_len=len(doc)
        max_num_tokens = max_seq_length - 3  #[CLS, SEQ,SEQ]
        target_seq_length=max_num_tokens
        if rng.random() < short_seq_prob:
            target_seq_length = rng.randint(2, max_num_tokens)
        a_start=0 
        while a_start<=doc_len-4:
            tokens=[] 
            segment_ids = []
            is_next_sentence=0
            a_length = rng.randint(1, target_seq_length - 1) # a 句子长度随机取
            a_end=a_start+a_length
            a_sentence=doc[a_start:a_end]
            b_length=target_seq_length-a_length # b的长度为句子长度- a句子长度
            if b_length== 1 or rng.random() < 0.5: # B句子是否来自下一句,有50%概率来自下一句
                for _ in range(10): # 从其他文档中随机选择一个文档，里的句子作为一个句子
                    random_document_index = rng.randint(0, len(all_documents) - 1)
                    if random_document_index != document_index:
                        break 
                random_document_next = all_documents[random_document_index]
                random_start = rng.randint(0, len(random_document_next) - 1)
                b_sentence=random_document_next[random_start:random_start+b_length]
                a_start=a_end # 下一次的a_start 从a_end开始
                is_next_sentence=0
            else:
                b_end=min(a_end+b_length,doc_len)
                b_sentence=doc[a_end:a_end+b_length]
                # 下一次的a_start 从a_length+b_length开始
                a_start+=target_seq_length
                is_next_sentence=1
            if  len(a_sentence) >= 1 and  len(b_sentence) >= 1:
                tokens.append("[CLS]")
                segment_ids.append(0)
                tokens.extend(a_sentence)
                [segment_ids.append(0) for i in a_sentence]
                tokens.append("[SEP]")
                segment_ids.append(0)
                
                tokens.extend(b_sentence)
                [segment_ids.append(1) for i in b_sentence]
                tokens.append("[SEP]")
                segment_ids.append(1)
                sentences.append((tokens,segment_ids,is_next_sentence))
    return sentences
sentences= getTrainSentenceFromDoc(all_documents)
print("".join(sentences[-1][0]))
print("is next sentence ",sentences[-1][2])
print("".join([str(i) for i in sentences[-1][1]]))

[CLS]下面將爲您介紹種植牙是如何進行分類的的相關內容，這將[SEP]會花費您約兩到三分鍾的時間進行浏覽，感謝您的閱讀：種植牙齒一起咬合作用強、不用磨牙齒、美觀大方舒服等優勢快速攻占牙齒治療銷售市場，可是種植牙齒的品種也是豐富多彩的，你對種植牙齒的類型有多少認識呢？種植牙齒的歸類：一、按其所需的栽植手術治療頻次分成一次嵌入牙齒種植體和二次嵌入牙齒種植體。二、按材質的做成形狀，種植牙齒的類別有圓柱體、葉狀、錨狀等牙齒種植體。三、按種植體在修補體構造中的部位分成分散端牙齒種植體、正中間牙齒種植體和全颌種植體。四、依據原材料，種植牙齒的類別有金屬材料牙齒種植體、瓷器類牙齒種植體、炭素類牙齒種植體、高分子材料高聚物牙齒種植體和複合材質牙齒種植體。五、一般較多地以種植體深層次人體機構的層面將其分成黏膜內牙齒種植體、骨膜下種植體、根管中牙齒種植體和骨內種植體等，這種全是種植牙齒的歸類。[SEP]
is next sentence  1
00000000000000000000000000001111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111


In [15]:
# 7.4 构建mask训练句子
max_predictions_per_seq=20
masked_lm_prob=0.2
class TrainingInstance:
    def __init__(self, tokens, segment_ids, masked_lm_positions, masked_lm_labels, is_random_next):
        self.tokens = tokens
        self.segment_ids = segment_ids
        self.is_random_next = is_random_next
        self.masked_lm_positions = masked_lm_positions
        self.masked_lm_labels = masked_lm_labels

    def __str__(self):
        s = ""
        s += "tokens: %s\n" % (" ".join(self.tokens))
        s += "segment_ids: %s\n" % (" ".join(map(str, self.segment_ids)))
        s += "is_random_next: %s\n" % self.is_random_next
        s += "masked_lm_positions: %s\n" % (" ".join(map(str, self.masked_lm_positions)))
        s += "masked_lm_labels: %s\n" % (" ".join(self.masked_lm_labels))
        s += "\n"
        return s

    def __repr__(self):
        return self.__str__()
		
def getMaskSentence(sentences):
    instances=[]
    for tokens,segment_ids,is_next_sentence in sentences:
        cand_indexes = []
        for (i, token) in enumerate(tokens):
            if token == "[CLS]" or token == "[SEP]":
                continue
            cand_indexes.append(i)
        rng.shuffle(cand_indexes)
        
        output_tokens = list(tokens)
        num_to_predict = min(max_predictions_per_seq, max(1, int(round(len(tokens) * masked_lm_prob))))
    
        masked_lms=[]
        for num in range(num_to_predict):
            index=cand_indexes[num]
            masked_token = None
            if rng.random() < 0.8:
                masked_token = "[MASK]"
            else:
                if rng.random() < 0.5:
                    masked_token = tokens[index]
                else:
                    masked_token = vocab_words[rng.randint(0, len(vocab_words) - 1)]
    
            output_tokens[index] = masked_token
    
            masked_lms.append((index, tokens[index]))
            
        masked_lms = sorted(masked_lms, key=lambda x: x[0])
        masked_lm_positions = []
        masked_lm_labels = []
        for p in masked_lms:
    #         print("p",p)
            masked_lm_positions.append(p[0])
            masked_lm_labels.append(p[1])
        
        instance=TrainingInstance(output_tokens,segment_ids,masked_lm_positions,masked_lm_labels,is_next_sentence)
        instances.append(instance)
    return instances
instances=getMaskSentence(sentences)
instances[0]

tokens: [CLS] 尽 管 淡 出 中 国 [MASK] 坛 好 多 年 ， 但 [UNK] 龙 哥 [UNK] 还 是 个 在 亚 足 联 甚 至 国 际 足 联 都 举 足 轻 重 的 人 物 [MASK] 他 现 在 头 上 那 个 [UNK] 中 [MASK] 足 协 副 主 席 [UNK] 的 头 衔 在 国 内 属 于 前 朝 遗 物 ， 他 目 前 是 [SEP] 田 亚 岐 [MASK] ， 自 1976 年 至 今 ， 已 在 秦 公 陵 [MASK] 内 发 现 了 49 座 大 墓 ， 但 这 49 座 大 墓 中 有 没 有 秦 [MASK] 公 的 墓 ？ 尚 待 进 一 步 的 考 古 发 现 予 以 确 认 。 泉 上 秦 伯 坟 ， 下 埋 三 良 [MASK] 。 三 良 百 夫 特 ， 岂 为 无 益 死 。 当 年 不 幸 见 迫 胁 ， 诗 人 尚 记 临 穴 惴 。 岂 如 田 横 海 中 客 [MASK] 中 原 [MASK] 汉 无 报 所 。 秦 国 吞 西 周 ， 康 公 穆 公 子 [MASK] 尽 力 事 康 公 ， 穆 公 不 为 负 。 岂 必 杀 身 従 之 游 ， 夫 子 乃 以 侯 嬴 所 为 疑 三 子 。 王 泽 既 未 竭 ， 君 子 不 为 诡 。 三 良 [UNK] 秦 穆 ， 要 自 不 [MASK] 已 。 苏 辙 （ 103 ##9 - 111 ##2 ） 字 子 由 ， [MASK] 州 眉 山 （ 今 属 四 川 ） 人 。 嘉 佑 二 年 （ 105 ##7 ） 与 其 兄 苏 轼 同 登 进 士 科 。 神 宗 朝 ， 为 制 置 三 司 条 例 司 属 官 。 因 反 对 [MASK] 安 石 变 法 ， 出 为 河 南 推 官 。 哲 宗 时 ， 召 为 秘 书 省 校 书 郎 。 元 佑 元 年 为 右 司 谏 ， 历 官 御 史 中 丞 、 尚 书 右 丞 、 门 下 侍 郎 。 后 因 事 忤 哲 宗 及 元 丰 诸 臣 ， 出 知 汝 州 、 再 [MASK] 雷 州 安 置 ， 移 循 州 。 徽 宗 立 ， 徙 永 州 、 岳 州 。 后 复 太 中 大 夫 ， 又 降 居 许 州 ， 致 仕 。 自 号 [MASK] 滨 遗 老 。 卒 ， 谥 文 定 

In [36]:
len(instances)

50

In [16]:
# 7.5 写入本地
def write_instance_to_json_files(instances, tokenizer, max_seq_length, max_predictions_per_seq, output_file):
    all_features = []

    for (inst_index, instance) in enumerate(instances):
        input_ids = tokenizer.convert_tokens_to_ids(instance.tokens)
        input_mask = [1] * len(input_ids)
        segment_ids = list(instance.segment_ids)

        assert len(input_ids) <= max_seq_length

        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        masked_lm_positions = list(instance.masked_lm_positions)
        masked_lm_ids = tokenizer.convert_tokens_to_ids(instance.masked_lm_labels)
        masked_lm_weights = [1.0] * len(masked_lm_ids)

        while len(masked_lm_positions) < max_predictions_per_seq:
            masked_lm_positions.append(0)
            masked_lm_ids.append(0)
            masked_lm_weights.append(0.0)

        next_sentence_label = 1 if instance.is_random_next else 0

        features = {
            "input_ids": input_ids,
            "input_mask": input_mask,
            "segment_ids": segment_ids,
            "masked_lm_positions": masked_lm_positions,
            "masked_lm_ids": masked_lm_ids,
            "masked_lm_weights": masked_lm_weights,
            "next_sentence_labels": next_sentence_label
        }

        all_features.append(features)


    with open(output_file, "w", encoding="utf-8") as writer:
        for feature in all_features:
            writer.write(json.dumps(feature, ensure_ascii=False) + "\n")

    print(f"Wrote {len(all_features)} total instances to {output_file}")
	

In [19]:
output_file="./data/bert_output_data3.json"

In [20]:
write_instance_to_json_files(instances, tokenizer, max_seq_length, max_predictions_per_seq, output_file)

Wrote 50 total instances to ./data/bert_output_data3.json
